# SVM Solution using Gradient Descent

**Uzair Ahmad**


In [44]:
import numpy as np
from sklearn.preprocessing import normalize

## The Gram Matrix:

the inner-product of feature vectors.
The inner products represent the similarity between feature vectors

In [48]:
X = np.array([[2, 2],[2, -1],[-2,-2],[-2,1]])
N = X.shape[0]
X = normalize(X)
print(X)
Xi_Xj = np.round(np.dot(X, X.T))
Xi_Xj

[[ 0.70710678  0.70710678]
 [ 0.89442719 -0.4472136 ]
 [-0.70710678 -0.70710678]
 [-0.89442719  0.4472136 ]]


array([[ 1.,  0., -1., -0.],
       [ 0.,  1., -0., -1.],
       [-1., -0.,  1.,  0.],
       [-0., -1.,  0.,  1.]])

## The class labels
The $M=Y_iY_j$ matrix represents the pairwise relationships between the class labels. Each element of $M$ indicates whether two corresponding elements in the original label vector $y$ belong to the same class (1) or different classes (-1).

In [49]:
y = np.array([1,1, -1,-1])

Yi_Yj = np.outer(y, y)
Yi_Yj

array([[ 1,  1, -1, -1],
       [ 1,  1, -1, -1],
       [-1, -1,  1,  1],
       [-1, -1,  1,  1]])

- By incorporating the class label information through the multiplication with $M$, the SVM's decision function may adapt to the pairwise relationships between classes, potentially leading to decision boundaries that better separate different classes in the feature space.

In [50]:
G = Yi_Yj * Xi_Xj
G

array([[1., 0., 1., 0.],
       [0., 1., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 0., 1.]])

 ## Initializtion
  - alphas: Initializes the Lagrange multipliers to zeros. These multipliers are the variables being optimized in the dual problem of the SVM.
  - ones: Creates an array of ones with the same size as the number of samples in the dataset.
  - lr: Defines the learning rate for the gradient ascent algorithm.
  - C: Represents the regularization parameter in the SVM formulation.

In [51]:
# Initialize alphas as zeros
alphas = np.zeros(N)

# Constants
ones = np.ones(N)
lr = 0.1
C = 1

## Gradient of Dual Formulation of SVM

Let's derive the gradient of the objective function with respect to the Lagrange multipliers in the context of the SVM dual optimization problem.

In the dual formulation of SVMs, the objective function to be maximized is typically expressed as follows:

$
L(\boldsymbol{\alpha}) = \sum_{i=1}^{N} \alpha_i - \frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_i \alpha_j y_i y_j K(\mathbf{x}_i, \mathbf{x}_j)
$

Where:
- $ \boldsymbol{\alpha} $ is the vector of Lagrange multipliers.
- $ N $ is the number of samples in the dataset.
- $ y_i $ and $ y_j $ are the class labels of samples $ i $ and $ j $.
- $ K(\mathbf{x}_i, \mathbf{x}_j) $ is the kernel function evaluating the inner product between feature vectors $ \mathbf{x}_i $ and $ \mathbf{x}_j$.

The gradient of the objective function $ L(\boldsymbol{\alpha}) $ with respect to the Lagrange multipliers $ \boldsymbol{\alpha} $ can be computed as follows:

$
\frac{\partial L(\boldsymbol{\alpha})}{\partial \alpha_i} = 1 - y_i \sum_{j=1}^{N} \alpha_j y_j K(\mathbf{x}_i, \mathbf{x}_j)
$

Where $ i = 1, 2, ..., N $.

Here's the derivation:

1. The derivative of the first term $ \sum_{i=1}^{N} \alpha_i $ with respect to $ \alpha_i $ is simply 1.

2. For the second term $ -\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_i \alpha_j y_i y_j K(\mathbf{x}_i, \mathbf{x}_j) $, we can focus on the part involving $ \alpha_i $. Taking the derivative with respect to $ \alpha_i $ yields:

$
\frac{\partial}{\partial \alpha_i} \left( -\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_i \alpha_j y_i y_j K(\mathbf{x}_i, \mathbf{x}_j) \right) = -y_i \sum_{j=1}^{N} \alpha_j y_j K(\mathbf{x}_i, \mathbf{x}_j)
$

This is because when $ j $ runs over all indices, only $ \alpha_i $ comes out as a constant, hence the summation is left over.

3. Finally, the total gradient is the sum of gradients of each term, leading to:

$
\frac{\partial L(\boldsymbol{\alpha})}{\partial \alpha_i} = 1 - y_i \sum_{j=1}^{N} \alpha_j y_j K(\mathbf{x}_i, \mathbf{x}_j)
$


## The main loop

In [52]:
# Main loop
for _ in range(10):
    # Compute the gradient
    gradient = ones - np.dot(Yi_Yj, alphas)

    # Update alphas
    alphas += lr * gradient

    # Clip alphas to be between 0 and C
    #alphas = np.clip(alphas, 0, C)
    alphas[alphas > C] = C
    alphas[alphas < 0] = 0
    # Compute the objective function
    gain = np.sum(alphas) - 0.5 * np.sum(np.outer(alphas, alphas) * G)
    print(gain)

0.36
0.64
0.8400000000000001
0.96
1.0
0.96
0.8400000000000001
0.6400000000000001
0.3600000000000003
4.440892098500626e-16


Identifying Support Vectors: extracts the indices of non-zero Lagrange multipliers, which correspond to the support vectors in the SVM solution.

In [53]:
# Get the indices of non-zero alphas (support vectors)
index = np.where(alphas > 0)[0]
index

array([0, 1, 2, 3])

In [54]:
Xi_Xs = np.dot(X, X[index].T)
Xi_Xs

array([[ 1.        ,  0.31622777, -1.        , -0.31622777],
       [ 0.31622777,  1.        , -0.31622777, -1.        ],
       [-1.        , -0.31622777,  1.        ,  0.31622777],
       [-0.31622777, -1.        ,  0.31622777,  1.        ]])

# The $w$s of decision boundary
To calculate the weights of the linear boundary in a linear SVM, we typically use the formula derived from the support vectors and their corresponding Lagrange multipliers.

In the dual formulation of linear SVMs, the decision boundary can be expressed as:

$f(x)=∑_{i=1}^Nα_iy_i⟨x_i,x⟩+b$

To calculate the weights ww of the linear boundary, you take the sum of the Lagrange multipliers αiyiαi​yi​ times their corresponding support vectors $x_i$​:

$w=\sum_{i=1}^Nα_iy_ix_i$

In code, this can be represented as:

In [55]:
# Calculate the weights of the linear boundary
w = np.dot((alphas * y)[index], X[index])
w

array([3.20306794, 0.51978637])

## The bias

Compute the mean of the differences between the class labels of support vectors and the similarities or distances of all samples to the support vectors. This mean value represents the bias term bb for the SVM decision function.

In [56]:
b_i = y[index] - (alphas * y).dot(np.dot(X, X[index].T))
b = np.mean(b_i)
b

0.0

# The decision boundary

While the expression itself is not explicitly the decision boundary equation, it represents the decision function that determines the boundary between classes in SVM classification. It evaluates the sign of the function to classify input vectors into respective classes.

```python
np.sign((alphas * y).dot(np.dot(X, X[index].T)) + b)
```

is computing the sign of the decision function. In SVMs, the decision function is defined as:

$ f(\mathbf{x}) = \text{sign}\left(\sum_{i=1}^{N} \alpha_i y_i \mathbf{x}_i, \mathbf{x} + b\right) $

where:
- $\alpha_i $ are the Lagrange multipliers.
- $ y_i $ are the class labels.
- $ \mathbf{x}_i, \mathbf{x} $ is the inner-product of features .
- $ b $ is the bias term.

Here's how the provided expression corresponds to the decision function:

1. **Computing Inner Product:**
   ```python
   (alphas * y).dot(np.dot(X, X[index].T))
   ```
   This part computes the inner product between the feature vectors $ \mathbf{x} $ and the support vectors, weighted by the Lagrange multipliers and class labels.

2. **Adding Bias:**
   ```python
   (alphas * y).dot(np.dot(X, X[index].T)) + b
   ```
   The bias term $ b $ is added to the result of the inner product. This adjusts the decision boundary based on the location of the support vectors and their corresponding class labels.

3. **Taking the Sign:**
   ```python
   np.sign((alphas * y).dot(np.dot(X, X[index].T)) + b)
   ```
   The `np.sign()` function returns -1 if the argument is negative, 0 if it's zero, and +1 if it's positive. This determines the class label assigned by the SVM for a given input vector $ \mathbf{x} $.

Therefore, the expression calculates the sign of the decision function, which determines the predicted class label for a given input vector $ \mathbf{x} $. If the result is positive, it means the input vector belongs to one class, and if it's negative, it belongs to the other class.


In [57]:
np.sign((alphas * y).dot(np.dot(X, X[index].T)) + b)

array([ 1.,  1., -1., -1.])